In [ ]:
%pip install azure-ai-projects
%pip install azure-core azure-identity
%pip install yfinance
%pip install opentelemetry opentelemetry-sdk azure-core-tracing-opentelemetry

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet


# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.getenv("AZAI_PROJECT_CONNECTION_STRING"),
)

In [2]:
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor

# Enable Azure Monitor tracing
application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your AI Foundry project page.")
    exit()
configure_azure_monitor(connection_string=application_insights_connection_string)

scenario = "WorkshopExercises"
tracer = trace.get_tracer(__name__)

In [3]:
from agent_tools import user_functions
# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

agent = project_client.agents.create_agent(
    model="gpt-4o-2", name="Finance Bot", 
    instructions="You are a stock trading assistant. You can help users with stock trading and financial analysis.", 
    toolset=toolset,
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_UzMhF3KcM7ksgQCUM5JKyzdL


In [4]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_jNQGOl1ElbhIaIBvv3Mpk7mB


In [5]:
@tracer.start_as_current_span(name="run_query")
def run(query):
    with tracer.start_as_current_span("run_query") as span:
        span.set_attribute("input.query", query)

        # Create message to thread
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=query
        )
        print(f"Created message, ID: {message.id}")

        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
        print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        result = messages.data[0]['content'][0].text.value
        print(f"Messages: {result}")

        span.set_attribute("output.result", result)
        return result

In [6]:
output = run("What is the current stock price of Top 10 Tech companies?")

Created message, ID: msg_5T0xfVECT0LYiNnpviaIBMvn
Run finished with status: completed
Messages: Here are the current stock prices of the top 10 tech companies:

1. **Apple (AAPL)**: $223.85
2. **Microsoft (MSFT)**: $390.58
3. **Alphabet (GOOGL)**: $162.24
4. **Amazon (AMZN)**: $201.36
5. **Meta Platforms (META)**: $602.58
6. **Tesla (TSLA)**: $273.13
7. **NVIDIA (NVDA)**: $111.43
8. **Oracle (ORCL)**: $145.78
9. **IBM (IBM)**: $246.21
10. **Adobe (ADBE)**: $396.15


In [7]:
# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")


Deleted agent


In [5]:
get_stock_price("BRK-B")

'{"stock_price": 534.52}'